In [ ]:
%pip install ipywidgets onnx onnxruntime-gpu transformers sentence-transformers optimum[onnxruntime] fastembed-gpu qdrant-client[fastembed-gpu] elasticsearch tqdm 

# $ optimum-cli export onnx --model intfloat/multilingual-e5-large --task feature-extraction models/multilingual-e5-large

#%pip install fastembed-gpu qdrant-client[fastembed-gpu] elasticsearch tqdm unstructured ipywidgets jsonlines pandas pyarrow numpy transformers torch onnx dateparser spacy[cuda12x] nvidia-ml-py3

In [2]:
import onnxruntime as ort
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")

# Load the ONNX model with GPU support
session = ort.InferenceSession(
    "models/multilingual-e5-large/model.onnx",
    providers=["CUDAExecutionProvider"]
)

print(session.get_providers())

['CPUExecutionProvider']


In [1]:
import onnxruntime as ort
print(ort.get_available_providers())


['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [1]:
from datasets import load_dataset
dataset = load_dataset('wikipedia', '20220301.nl', split='train[:1000]')
texts = dataset['text']

/home/bizon/Development/bron-embeddings/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import time
import numpy as np
import onnxruntime as ort
# from datasets import load_dataset
from transformers import AutoTokenizer
import concurrent.futures

# Load the dataset
# texts = dataset['text']

texts = ["Zoek regelingen op overheid.nl Ziet u een fout in deze regeling? Meld het ons op regelgeving@overheid.nl! Bergen (NH) beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016 Wetstechnische informatie Organisatie Bergen (NH) Organisatietype Gemeente Officiële naam regeling beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016 Citeertitel beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016 Vastgesteld door college van burgemeester en wethouders Onderwerp maatschappelijke zorg en welzijn Eigen onderwerp Opmerkingen met betrekking tot de regeling Geen Wettelijke grondslag(en) of bevoegdheid waarop de regeling is gebaseerd Participatiewet, art. 35 Regelgeving die op deze regeling is gebaseerd (gedelegeerde regelgeving) Geen Datum inwerkingtreding Terugwerkende kracht tot en met Datum uitwerkingtreding Betreft Datum ondertekening Bron bekendmaking Kenmerk voorstel 16-06-2016 01-01-2016 nieuwe regeling 10-05-2016 Gemeenteblad, 15 juni 2016, nr. 72683 Onbekend Tekst van de regeling Intitulé beleidsregels meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016 Het college van de gemeente Bergen Gelet op artikel 35 van de Participatiewet; Besluit vast te stellen de beleidsregels ‘meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016’ Artikel 1 Begripsbepalingen 1. Alle begrippen die in deze beleidsregel worden gebruikt en die niet nader worden omschreven hebben dezelfde betekenis als in de Participatiewet, de Algemene wet bestuursrecht (Awb), de Gemeentewet, de Algemene wet inkomensafhankelijke regelingen (Awir) en de eerder vastgestelde beleidsregels bijzondere bijstand. 2. In deze beleidsregel wordt verstaan onder: a. het college: het college van burgemeester en wethouders; b. de wet: de Participatiewet; c. bijzondere bijstand: de bijstand bedoeld in artikel 35 Participatiewet; d. meerkosten: het gaat bij meerkosten niet om alle kosten, maar om de extra kosten die boven de gewone kosten uitkomen in relatie tot de chronische aandoening. Het betreft niet alleen directe ziekte- of zorgkosten, maar ook indirecte kosten die een gevolg kunnen zijn en samenhangen met de chronische aandoening. Als uitgangspunt voor gewone kosten kunnen de normen van het Nationaal Instituut voor Budgetvoorlichting (Nibud) gelden; e. draagkracht: het gedeelte van het inkomen of vermogen dat de inwoner geacht wordt aan te wenden om in de bijzondere kosten te voorzien; f. draagkrachtperiode: de periode waarover de financiële draagkracht van de inwoner wordt vastgesteld; g. inkomen: het inkomen zoals bedoeld in de artikel 31 tot en met 33 Participatiewet; h. minimum inkomen: een inkomen van ten hoogste 120% van de voor de inwoner op datum aanvraag geldende bijstandsnorm; i. vermogen: het vermogen als bedoeld in artikel 34 lid 1 van de Participatiewet; j. bescheiden vermogen: een vermogen van maximaal het in artikel 34 lid 3 van de Participatiewet genoemde bedrag; k. voorliggende voorziening: de voorziening als bedoeld in artikel 15 Participatiewet; l. woning: een woning zoals bedoeld in artikel 1 onderdeel j Wet op de huurtoeslag, alsmede een woonwagen of woonschip, zoals bedoeld in artikel 3 lid 6 Participatiewet. Artikel 2 Doelstelling Deze beleidsregels regelen bijzondere bijstand ten behoeve van chronisch zieken en/of mensen met een chronische beperking met een minimum inkomen, zodat de meerkosten genoemd onder artikel 9 en artikel 10 geen belemmering vormen om zelfredzaam te zijn en maatschappelijk te participeren. Artikel 3 Definitie chronische ziekte en/of chronische beperking We spreken van een inwoner met een chronische ziekte of chronische beperking als die inwoner: a. het verplicht eigen risico van de zorgverzekering twee jaar achtereenvolgens voorafgaand aan de aanvraag volledig heeft opgemaakt; of b. het eigen risico van de zorgverzekering van het jaar voorafgaand aan de aanvraag en het lopende jaar volledig heeft opgemaakt; of c. beschikt over een gehandicaptenparkeerkaart; of d. een indicatie heeft voor de Wet langdurige zorg voor een periode langer dan 6 maanden; of e. een indicatie heeft voor de Wet maatschappelijke ondersteuning voor een periode langer dan 6 maanden; of f. op grond van de Participatiewet of Wajong ontheffing heeft gekregen van de arbeidsverplichting om medische redenen voor een periode langer dan 6 maanden; of g. een volledige WAO, WIA of Wajong uitkering heeft. Artikel 4 Kostendelersnorm 1. De kostendelersnorm als bedoeld in artikel 22a van de wet is van toepassing op deze beleidsregels. 2. De beleidsregels kostendelersnorm en verlagingen 2015 gemeente Bergen zijn van toepassing op aanvragen bijzondere bijstand. Artikel 5 Voorliggende voorzieningen Indien de inwoner gebruik kan maken van een andere voorziening die voorziet in de meerkosten zoals benoemd in artikel 9 en artikel 10, dan wordt deze voorziening als voorliggende voorziening gezien. Er kan dan geen aanspraak worden gemaakt op deze regeling 1. Als voorliggende voorziening als bedoeld in artikel 15 van de wet worden onder meer aangemerkt: a. de aanvullende verzekering van de Collectieve Zorgverzekering Minima (CZM); b. de Wet maatschappelijke ondersteuning 2015 (Wmo 2015); c. de Wet langdurige zorg (Wlz); d. de Zorgverzekeringswet (Zvw); e. de Algemene kinderbijslagwet (Akw); f. Basisverzekerering Zorg 2. De in het eerste lid genoemde voorliggende voorzieningen betreffen geen limitatieve opsomming. Artikel 6 De wijze en het tijdstip van aanvragen 1. Een aanvraag wordt met het daarvoor bestemde aanvraagformulier ingediend bij het college. Op het aanvraagformulier staat vermeld welke bewijsstukken belanghebbende bij de aanvraag moet overleggen. 2. een aanvraag dient te worden ingediend: a. voor belanghebbende die een beroep doet op artikel 9 van deze beleidsregels binnen de daarvoor geldende termijnen zoals benoemd in het vastgestelde beleid bijzondere bijstand. b. voor belanghebbende die een beroep doet op artikel 10 van deze beleidsregels binnen 4 weken na ontvangst van de specificatie waaruit blijkt hoe hoog het eigen aandeel is. Dit met uitzondering van de maanden januari 2016 tot en met mei 2016. Deze mogen met terugwerkende kracht in een keer worden ingediend in juni 2016. Artikel 7 Draagkracht 1. Bij de berekening van de draagkracht wordt uitgegaan van hetgeen van de beleidsregels bijzondere bijstand 2015 gemeente Bergen, vastgesteld op 10 september 2015. 2. Bij de berekening van de draagkracht wordt in ieder geval rekening gehouden met de volgende aftrekbare kosten, voor zover dit niet in het eigen beleid zoals genoemd onder lid 1 is geregeld. a. eigen bijdrage CAK; b. eigen aandeel schoonmaakondersteuning; c. kosten die worden gemaakt vanwege het niet kunnen deelnemen aan de CZM; d. kosten die worden gemaakt door het niet kunnen krijgen van kwijtscheldingen gemeentelijke belastingen en waterschapsbelastingen; e. kosten die worden gemaakt door het niet kunnen krijgen van huur- en zorgtoeslag. Artikel 8 Doelgroep 1. Voor bijzondere bijstand op grond van deze regeling komt in aanmerking de inwoner die: a. ingeschreven staat in de gemeente Bergen op datum van de aanvraag; en b. op de datum van aanvraag een inkomen heeft dat niet hoger is dan 120% van de van toepassing zijnde bijstandsnorm (dit betreft geen harde grens vanwege de aanvullingen van de draagkrachtberekening in artikel 7. Inwoners die boven de grens van het sociaal minimum leven kunnen vanwege meerkosten en het mislopen van deelname aan bepaalde regelingen dusdanige kosten maken dat zij onder de 120% uitkomen (oprekken draagkracht). Hierdoor kunnen ook zij een risico gaan lopen in zelfredzaamheid en participatie); en c. de inwoner die op de datum van aanvraag een vermogen heeft dat niet hoger is dan artikel 34 van de Participatiewet; en d. de inwoner die chronisch ziek is of een chronische beperking heeft zoals omschreven in artikel 3 van de beleidsregels; en e. de inwoner die de Nederlandse nationaliteit heeft of daaraan gelijkgesteld 2. Indien de inwoner in een andere gemeente al een vergoeding heeft verkregen voor de meerkosten als gevolg van de chronische ziekte of chronische beperking dan heeft de inwoner geen recht op een financiële vergoeding in de huidige gemeente. Artikel 9 (Para)medische kosten voor chronisch zieken en chronische beperking conform deze beleidsregels 1. Voor bijzondere bijstand komen de volgende (para)medische noodzakelijke kosten in aanmerking, indien er geen voorliggende voorziening is: a. de meerkosten van een medisch noodzakelijk dieet of aangepast voedsel; b. de ten gevolge van langdurige ziekte of beperking ontstane meerkosten van slijtage van kleding en/of beddengoed; c. de ten gevolge van langdurige ziekte of beperking ontstane meerkosten van bewassing van kleding en/of beddengoed; d. de meerkosten van verwarming ten gevolge van een beperking of langdurige ziekte; e. zelfzorggeneesmiddelen; f. pedicurekosten; g. vervoerskosten die buiten de regiotaxi vallen, maar wel (tijdelijk) noodzakelijk zijn; h. aanvullende verzekeringspremie voor chronisch zieken, die om gegronde redenen niet over (kunnen) stappen naar de CZM/gemeentepolis. 2. De in het eerste lid genoemde kosten betreffen geen limitatieve opsomming. Artikel 10 Eigen aandeel schoonmaakondersteuning 2016 De inwoner die gebruik maakt van de algemene voorziening schoonmaakondersteuning die voldoet aan de criteria in artikel 7 draagkracht komt in aanmerking voor bijzondere bijstand voor de volledige kosten van het eigen aandeel voor maximaal 8 uren per vier weken. Artikel 11 Inwerkingtreding en citeertitel 1. Deze beleidsregels treden met terugwerkende kracht in werking vanaf 1 januari 2016. Dit met uitzondering van artikel 9. 2. Artikel 9 van deze beleidsregels treedt in werking op de dag na bekendmaking tot een jaar daarna. 3. Deze beleidsregels kunnen worden aangehaald als: beleidsregels ‘meerkosten chronisch zieken en mensen met een chronische beperking gemeente Bergen 2016’."]

texts *= 1000  # Adjust the multiplier as needed

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large")

# Define batch size
batch_size = 32  # Adjust based on your GPU memory

# Split texts for two GPUs
texts_gpu0 = texts[:len(texts)//2]
texts_gpu1 = texts[len(texts)//2:]

# Initialize ONNX Runtime sessions for both GPUs
# providers_gpu0 = [('TensorrtExecutionProvider', {
#     'device_id': 0,
#     'trt_max_workspace_size': 1 << 30
# }), ('CUDAExecutionProvider', {'device_id': 0}), 'CPUExecutionProvider']

providers_gpu0 = [('CUDAExecutionProvider', {'device_id': 0}), 'CPUExecutionProvider']
session_gpu0 = ort.InferenceSession(
    "models/multilingual-e5-large/model.onnx",
    providers=providers_gpu0
)

# providers_gpu1 = [('TensorrtExecutionProvider', {
#     'device_id': 1,
#     'trt_max_workspace_size': 1 << 30
# }), ('CUDAExecutionProvider', {'device_id': 1}), 'CPUExecutionProvider']
providers_gpu1 = [('CUDAExecutionProvider', {'device_id': 1}), 'CPUExecutionProvider']
session_gpu1 = ort.InferenceSession(
    "models/multilingual-e5-large/model.onnx",
    providers=providers_gpu1
)

print(session_gpu0.get_providers())
print(session_gpu1.get_providers())

def process_batches(texts, session, device_id):
    embeddings_list = []
    total_inference_time = 0.0
    num_batches = len(texts) // batch_size + int(len(texts) % batch_size > 0)
    
    for i in range(num_batches):
        batch_texts = texts[i * batch_size:(i + 1) * batch_size]
        
        # Tokenize
        inputs = tokenizer(
            batch_texts,
            return_tensors="np",
            padding=True,
            truncation=True,
        )
        
        feed = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
        }
        
        # Time the inference
        start = time.time()
        outputs = session.run(None, feed)
        embeddings = outputs[0]
        end = time.time()
        
        inference_time = end - start
        total_inference_time += inference_time
        
        embeddings_list.append(embeddings)
        
        print(f"Device {device_id}: Processed batch {i+1}/{num_batches} in {inference_time:.4f} s")
    
    # Combine embeddings
    embeddings_array = np.concatenate(embeddings_list, axis=0)
    
    return embeddings_array, total_inference_time

# Define batch size
batch_size = 32  # Adjust based on your GPU memory

# Split texts for two GPUs
texts_gpu0 = texts[:len(texts)//2]
texts_gpu1 = texts[len(texts)//2:]

# Start parallel processing
start_time = time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    future_gpu0 = executor.submit(process_batches, texts_gpu0, session_gpu0, 0)
    future_gpu1 = executor.submit(process_batches, texts_gpu1, session_gpu1, 1)
    
    embeddings_gpu0, time_gpu0 = future_gpu0.result()
    embeddings_gpu1, time_gpu1 = future_gpu1.result()

# Combine embeddings from both GPUs
all_embeddings = np.concatenate([embeddings_gpu0, embeddings_gpu1], axis=0)
total_inference_time = max(time_gpu0, time_gpu1)  # Since they run in parallel
end_time = time.time()

print(f"Total inference time: {end_time - start_time:.4f} s")
print(f"Total samples processed: {len(all_embeddings)}")


['CUDAExecutionProvider', 'CPUExecutionProvider']
['CUDAExecutionProvider', 'CPUExecutionProvider']
Device 0: Processed batch 1/16 in 0.3234 s
Device 1: Processed batch 1/16 in 0.3563 s
Device 0: Processed batch 2/16 in 0.3349 s
Device 1: Processed batch 2/16 in 0.3344 s
Device 0: Processed batch 3/16 in 0.2998 s
Device 1: Processed batch 3/16 in 0.3015 s
Device 0: Processed batch 4/16 in 0.2990 s
Device 1: Processed batch 4/16 in 0.3004 s
Device 0: Processed batch 5/16 in 0.2991 s
Device 1: Processed batch 5/16 in 0.3028 s
Device 0: Processed batch 6/16 in 0.2991 s
Device 1: Processed batch 6/16 in 0.3014 s
Device 0: Processed batch 7/16 in 0.2991 s
Device 1: Processed batch 7/16 in 0.3060 s
Device 0: Processed batch 8/16 in 0.2995 s
Device 1: Processed batch 8/16 in 0.3001 s
Device 0: Processed batch 9/16 in 0.3031 s
Device 1: Processed batch 9/16 in 0.3003 s
Device 0: Processed batch 10/16 in 0.2993 s
Device 1: Processed batch 10/16 in 0.3009 s
Device 0: Processed batch 11/16 in 0.2

In [3]:
texts = ["First text.", "Second text."]
inputs = tokenizer(
    texts,
    return_tensors="np",
    padding=True,
    truncation=True,
)

feed = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"],
}

# time the inference
import time
start = time.time()
outputs = session_trt.run(None, feed)
embeddings = outputs[0]
end = time.time()
print(f"Time TRT: {end - start:.4f} s")

# time the inference
import time
start = time.time()
outputs = session.run(None, feed)
embeddings = outputs[0]
end = time.time()
print(f"Time CUDA: {end - start:.4f} s")



print(session.get_providers())
print(embeddings)


Time TRT: 0.0253 s
Time CUDA: 0.0230 s
['CPUExecutionProvider']
[[[ 0.16768321  0.53447413 -0.064367   ...  0.32083124 -0.7574888
    0.23727623]
  [ 0.19932368  0.38911384  0.16801043 ...  0.32452258 -0.4775465
   -0.27104193]
  [-0.01343609  0.5164142   0.27062693 ...  0.31079036 -0.27674928
    0.19128618]
  [ 0.00662065  0.7551768   0.08567598 ...  0.3852061  -0.385394
   -0.22701138]
  [ 0.16747251  0.5345489  -0.06415916 ...  0.32060975 -0.75732356
    0.23722246]]

 [[ 0.37464076  0.15283921 -0.3183998  ...  0.60188234 -0.50762534
    0.33668876]
  [ 0.32599247  0.32943702 -0.05340838 ...  0.5908763  -0.05620044
   -0.23831595]
  [ 0.17811598  0.26987872 -0.19269386 ...  0.3972184  -0.4696718
    0.16088402]
  [ 0.34185344  0.548715   -0.19215056 ...  0.7062544  -0.5366848
   -0.07479396]
  [ 0.3747324   0.15315871 -0.3181942  ...  0.601699   -0.5075573
    0.33677804]]]


In [ ]:

texts = ["First text.", "Second text."]
inputs = tokenizer(
    texts,
    return_tensors="np",
    padding=True,
    truncation=True,
)

feed = {
    "input_ids": inputs["input_ids"],
    "attention_mask": inputs["attention_mask"],
}
# time the inference
import time
start = time.time()
outputs = session.run(None, feed)
embeddings = outputs[0]
end = time.time()
print(f"Time: {end - start:.3f} s")


print(session.get_providers())
print(embeddings)


In [1]:
import onnxruntime as ort
print(ort.get_available_providers())


['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [1]:
import torch
torch.cuda.empty_cache()
